Modèle de réseaux de neurones sur le dataset du Joueur haut - Henri, Minh, Maxime
- Coup droit
- Déplacement
- Revers
- Service

In [72]:
import numpy as np
import cv2
import os
from smart.video import Video, Image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import tensorflow as tf
import random
from sklearn.preprocessing import LabelEncoder

In [73]:
# Configuration
path_dataset = 'img/JHaut'

# Réseau de neurones classique

## Prises des frames et des vidéos

In [74]:
list_videos = []

In [75]:
y = []
all_paths = []
for dirpath, dirnames, _ in os.walk(path_dataset):
    for dir_type in dirnames:
        
        path_fol = os.path.join(dirpath, dir_type)
        for file in os.listdir(path_fol):
            path_fol_img = os.path.join(path_fol, file)
            if os.path.isdir(path_fol_img) and file == 'images':
                frames = []
                for file_img in os.listdir(path_fol_img):
                    path_img = os.path.join(path_fol_img, file_img)
                    img_obj = Image.load_image(cv2.IMREAD_GRAYSCALE, path_img)
                    img = img_obj.img
                    frames.append(img)	
                if (len(frames) == 15):
                    output_res = path_img.split('\\')[1]
                    all_paths.append(path_fol_img)
                    vid = Video.read_video_from_frames(frames)
                    list_videos.append(vid)
                    y.append(output_res)

In [76]:
list_videos

In [77]:
# Output sans label encoder
y

['coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'service',
 'service',
 'service',
 'service',
 'service',
 'service',
 'service',
 'service',
 'service',
 'service',
 'service',
 'service',
 'service']

In [78]:
# One-hot encoding
encoder = LabelEncoder()

Transformer output y en one hot vector
- 0: coup droit
- 1: déplacement
- 2: revers
- 3: service

In [79]:
y = encoder.fit_transform(y)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3], dtype=int64)

In [80]:
output_y = np.unique(y)
output_y

array([0, 1, 2, 3], dtype=int64)

In [81]:
# Résolution des images
input_shape_model = list_videos[0].frames.shape[0] * list_videos[0].frames.shape[1] * list_videos[0].frames.shape[2]
input_shape_model

37500

In [82]:
len(list_videos)

67

In [83]:
train_size = int(len(list_videos) * 0.8)
test_size = len(list_videos) - train_size

In [84]:
## Split video train, test
random.seed(42)
vids_train = random.sample(list_videos, k = train_size)
vids_test = [vid for vid in list_videos if vid not in vids_train]
y_train = [y[i] for i in range(len(y)) if list_videos[i] in vids_train]
y_test = [y[i] for i in range(len(y)) if list_videos[i] in vids_test]
paths_test = [all_paths[i] for i in range(len(all_paths)) if list_videos[i] in vids_test]
len(vids_train), len(y_train), len(vids_test), len(y_test)

(53, 53, 14, 14)

In [85]:
paths_test

['img/JHaut\\coup droit\\AS_18386\\images',
 'img/JHaut\\coup droit\\AS_29198\\images',
 'img/JHaut\\coup droit\\car5493-5847138\\images',
 'img/JHaut\\deplacement\\17694-1786278\\images',
 'img/JHaut\\deplacement\\1980-2563441\\images',
 'img/JHaut\\deplacement\\28040-28230129\\images',
 'img/JHaut\\deplacement\\44260-4450099\\images',
 'img/JHaut\\deplacement\\6481-660086\\images',
 'img/JHaut\\deplacement\\AS_15180\\images',
 'img/JHaut\\revers\\AS_34270\\images',
 'img/JHaut\\revers\\car5936-6390142\\images',
 'img/JHaut\\revers\\car5936-6390257\\images',
 'img/JHaut\\service\\AS_19185\\images',
 'img/JHaut\\service\\AS_35549\\images']

In [86]:
y_train = np.array(y_train, dtype=int)
y_test = np.array(y_test, dtype=int)
y_train

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3])

In [87]:
# Split input X en train, test set
X_train = np.zeros((train_size, 50*50*15), dtype=int)
for i, vid in enumerate(vids_train):
    seq_img = vid.frames.flatten()
    X_train[i] = seq_img
X_train.shape

(53, 37500)

In [88]:
X_test = np.zeros((test_size, 50*50*15), dtype=int)
for i, vid in enumerate(vids_test):
    seq_img = vid.frames.flatten()
    X_test[i] = seq_img
X_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [89]:
# Normalisation
X_train = X_train / 255.0
X_test = X_test / 255.0

## Modèle de réseau de neurones avec classification multiclass

In [90]:
tf.random.set_seed(1234) # for consistent results
model = Sequential(
    [               
        tf.keras.Input(shape = input_shape_model), # 50*50 * 15 = 37500
        Dense(units=16, activation="relu"),
        Dense(units=32, activation="relu"),
        Dense(units=64, activation="sigmoid"),
        Dense(units=128, activation="relu"),
        Dense(units=len(output_y), activation="softmax"),
    ]
)          
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 16)                600016    
                                                                 
 dense_16 (Dense)            (None, 32)                544       
                                                                 
 dense_17 (Dense)            (None, 64)                2112      
                                                                 
 dense_18 (Dense)            (None, 128)               8320      
                                                                 
 dense_19 (Dense)            (None, 4)                 516       
                                                                 
Total params: 611,508
Trainable params: 611,508
Non-trainable params: 0
_________________________________________________________________


In [91]:
checkpoint_name = 'checkpoints/Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [92]:
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = Adam(learning_rate=0.001), # 'Adam',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs = 100,
    batch_size = 32, # données transmises pour une session
    validation_split=0.2,
    callbacks=callbacks_list
)

Epoch 1/100
1/2 [==============>...............] - ETA: 0s - loss: 1.6034 - accuracy: 0.0000e+00
Epoch 1: val_loss improved from inf to 1.63703, saving model to checkpoints\Weights-001--1.63703.hdf5
2/2 [==============================] - 0s 143ms/step - loss: 1.5686 - accuracy: 0.0714 - val_loss: 1.6370 - val_accuracy: 0.0000e+00
Epoch 2/100
1/2 [==============>...............] - ETA: 0s - loss: 1.3144 - accuracy: 0.2812
Epoch 2: val_loss did not improve from 1.63703
2/2 [==============================] - 0s 33ms/step - loss: 1.2922 - accuracy: 0.3333 - val_loss: 2.2306 - val_accuracy: 0.0000e+00
Epoch 3/100
1/2 [==============>...............] - ETA: 0s - loss: 1.1354 - accuracy: 0.3750
Epoch 3: val_loss did not improve from 1.63703
2/2 [==============================] - 0s 32ms/step - loss: 1.1150 - accuracy: 0.4524 - val_loss: 2.7827 - val_accuracy: 0.0000e+00
Epoch 4/100
1/2 [==============>...............] - ETA: 0s - loss: 1.0569 - accuracy: 0.4375
Epoch 4: val_loss did not impro

In [25]:
# Save model
model.save('saved_models\classic_model_1_joueur_haut.h5')

In [93]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 29ms/step - loss: 2.4916 - accuracy: 0.5000


[2.4915664196014404, 0.5]

In [94]:
pred = model.predict(X_test)
pred

1/1 [==============================] - 0s 39ms/step


array([[9.3768126e-01, 5.8769848e-02, 3.3202462e-03, 2.2866404e-04],
       [1.4789995e-02, 9.7630125e-01, 8.7563572e-03, 1.5232492e-04],
       [4.2683479e-01, 5.6289965e-01, 9.7612320e-03, 5.0437305e-04],
       [3.4849536e-02, 8.5830021e-01, 1.0647215e-01, 3.7820748e-04],
       [2.6128751e-01, 7.3313445e-01, 5.2384809e-03, 3.3956187e-04],
       [1.3584740e-02, 9.7620732e-01, 1.0067902e-02, 1.4004315e-04],
       [6.7455277e-02, 9.2993563e-01, 2.4244040e-03, 1.8471872e-04],
       [1.5410262e-02, 9.8336315e-01, 1.1361671e-03, 9.0343259e-05],
       [9.8351246e-01, 1.5533330e-02, 8.4053824e-04, 1.1367043e-04],
       [2.5411665e-02, 9.5028055e-01, 2.4057344e-02, 2.5041259e-04],
       [5.9056103e-02, 9.2747504e-01, 1.3100634e-02, 3.6827565e-04],
       [4.2315223e-03, 1.8263814e-01, 8.1289792e-01, 2.3245366e-04],
       [4.4910014e-03, 9.9560238e-02, 8.9574826e-01, 2.0048725e-04],
       [7.4567511e-03, 8.4417129e-01, 1.4810446e-01, 2.6744147e-04]],
      dtype=float32)

In [95]:
y_pred = np.argmax(pred, axis=1)
print("Predicted labels: ", y_pred)
print("True labels:      ", y_test)

Predicted labels:  [0 1 1 1 1 1 1 1 0 1 1 2 2 1]
True labels:       [0 0 0 1 1 1 1 1 1 2 2 2 3 3]


**Accuracy: $7/14$ ($50 \%$)**

# Réseaux de neurones convolutifs

## Prises des frames et des vidéos

In [96]:
list_videos_cnn = []
y_cnn = []
for dirpath, dirnames, _ in os.walk(path_dataset):
    for dir_type in dirnames:
        
        path_fol = os.path.join(dirpath, dir_type)
        for file in os.listdir(path_fol):
            path_fol_img = os.path.join(path_fol, file)
            if os.path.isdir(path_fol_img) and file == 'images':
                frames = []
                for file_img in os.listdir(path_fol_img):
                    path_img = os.path.join(path_fol_img, file_img)
                    img_obj = Image.load_image(cv2.IMREAD_COLOR, path_img)
                    img = img_obj.img
                    frames.append(img)	
                if (len(frames) == 15):
                    output_res = path_img.split('\\')[1]
                    vid = Video.read_video_from_frames(frames)
                    list_videos_cnn.append(vid)
                    y_cnn.append(output_res)

In [97]:
list_videos_cnn[0].frames.shape[1:]

(50, 50, 3)

In [98]:
# One-hot encoding
encoder = LabelEncoder()

Transformer output y en one hot vector
- 0: coup droit
- 1: déplacement
- 2: revers
- 3: service

In [99]:
y_cnn = encoder.fit_transform(y_cnn)
y_cnn

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3], dtype=int64)

In [100]:
output_y = np.unique(y_cnn)
output_y

array([0, 1, 2, 3], dtype=int64)

In [101]:
len(list_videos_cnn)

67

In [102]:
train_size = int(len(list_videos_cnn) * 0.8)
test_size = len(list_videos_cnn) - train_size

In [104]:
## Split video train, test
random.seed(42)
vids_train_cnn = random.sample(list_videos_cnn, k = train_size)
vids_test_cnn = [vid for vid in list_videos_cnn if vid not in vids_train_cnn]
y_train_cnn = [y_cnn[i] for i in range(len(y_cnn)) if list_videos_cnn[i] in vids_train_cnn]
y_test_cnn = [y_cnn[i] for i in range(len(y_cnn)) if list_videos_cnn[i] in vids_test_cnn]
len(vids_train_cnn), len(y_train_cnn), len(vids_test_cnn), len(y_test_cnn)

(53, 53, 14, 14)

In [105]:
y_train_cnn = np.array(y_train_cnn, dtype=int)
y_test_cnn = np.array(y_test_cnn, dtype=int)
y_train_cnn

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3])

In [106]:
# Split input X en train, test set
X_train_cnn = np.zeros((train_size, 50, 15*50, 3), dtype=int)
for i, vid in enumerate(vids_train_cnn):
    for j, frame in enumerate(vid.frames):
        X_train_cnn[i, :, 50*j:50*(j+1), :] = frame
X_train_cnn.shape

(53, 50, 750, 3)

In [107]:
X_test_cnn = np.zeros((test_size, 50, 15*50, 3), dtype=int)
for i, vid in enumerate(vids_test_cnn):
    for j, frame in enumerate(vid.frames):
        X_test_cnn[i, :, 50*j:50*(j+1), :] = frame
X_test_cnn.shape

(14, 50, 750, 3)

In [108]:
input_shape_cnn = X_train_cnn.shape[1:]
input_shape_cnn

(50, 750, 3)

## Modèle de réseau de neurones convolutifs

In [141]:
model_cnn = Sequential(
    [
        tf.keras.Input(shape = input_shape_cnn), # 50, 15*50, 3
        Conv2D(filters=16, kernel_size=(3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Conv2D(filters=32, kernel_size=(3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        # Conv2D(filters=64, kernel_size=(3, 3), activation="relu"),
        # BatchNormalization(),
        # MaxPooling2D(pool_size=(2, 2)),
        # Dropout(0.2),
        
        Flatten(),
        # Dense(units=128, activation="relu"),
        Dense(units=len(output_y), activation="softmax"),
    ]
)

model_cnn.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 48, 748, 16)       448       
                                                                 
 batch_normalization_12 (Bat  (None, 48, 748, 16)      64        
 chNormalization)                                                
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 24, 374, 16)      0         
 g2D)                                                            
                                                                 
 dropout_12 (Dropout)        (None, 24, 374, 16)       0         
                                                                 
 conv2d_13 (Conv2D)          (None, 22, 372, 32)       4640      
                                                                 
 batch_normalization_13 (Bat  (None, 22, 372, 32)    

In [142]:
checkpoint_name = 'checkpoints/Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [143]:
model_cnn.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=Adam(learning_rate=0.001), # 'Adam',
    metrics=['accuracy']
)

history_cnn = model_cnn.fit(
    X_train_cnn, y_train_cnn,
    epochs = 8,
    batch_size = 32, # données transmises pour une session
    validation_split=0.2,
    callbacks=callbacks_list
)

Epoch 1/8
2/2 [==============================] - ETA: 0s - loss: 3.7184 - accuracy: 0.3095
Epoch 1: val_loss improved from inf to 362.18332, saving model to checkpoints\Weights-001--362.18332.hdf5
2/2 [==============================] - 1s 248ms/step - loss: 3.7184 - accuracy: 0.3095 - val_loss: 362.1833 - val_accuracy: 0.0000e+00
Epoch 2/8
2/2 [==============================] - ETA: 0s - loss: 9.2877 - accuracy: 0.5238
Epoch 2: val_loss did not improve from 362.18332
2/2 [==============================] - 0s 125ms/step - loss: 9.2877 - accuracy: 0.5238 - val_loss: 372.9922 - val_accuracy: 0.0000e+00
Epoch 3/8
2/2 [==============================] - ETA: 0s - loss: 0.4647 - accuracy: 0.8810
Epoch 3: val_loss did not improve from 362.18332
2/2 [==============================] - 0s 131ms/step - loss: 0.4647 - accuracy: 0.8810 - val_loss: 403.5988 - val_accuracy: 0.0000e+00
Epoch 4/8
2/2 [==============================] - ETA: 0s - loss: 2.3565 - accuracy: 0.7381
Epoch 4: val_loss did not i

In [144]:
model_cnn.evaluate(X_test_cnn, y_test_cnn)

1/1 [==============================] - 0s 60ms/step - loss: 46.2811 - accuracy: 0.3571


[46.281131744384766, 0.3571428656578064]

In [120]:
pred_cnn = model_cnn.predict(X_test_cnn)
pred_cnn

1/1 [==============================] - 0s 111ms/step


array([[1.6808382e-05, 9.9998319e-01, 1.4623816e-11, 0.0000000e+00],
       [4.3282096e-20, 4.9459021e-13, 1.0000000e+00, 0.0000000e+00],
       [7.4852690e-02, 9.2514735e-01, 1.0751601e-16, 0.0000000e+00],
       [7.3618253e-07, 9.9999928e-01, 3.0678612e-11, 2.4728244e-29],
       [9.9999917e-01, 7.7741157e-07, 3.0447034e-28, 4.5529375e-38],
       [1.0000000e+00, 2.2580164e-09, 5.6281295e-12, 0.0000000e+00],
       [1.0000000e+00, 2.1196392e-18, 3.1410056e-22, 0.0000000e+00],
       [9.9809319e-01, 1.9067571e-03, 3.6028285e-20, 4.8427297e-31],
       [1.6480216e-03, 9.9835205e-01, 4.5297893e-16, 1.0144309e-28],
       [5.4310608e-01, 4.5689389e-01, 1.7405993e-12, 1.2257344e-24],
       [5.4348352e-06, 9.9999452e-01, 1.1249352e-08, 3.2613151e-16],
       [8.9860947e-08, 8.4770671e-20, 9.9999988e-01, 4.1639471e-31],
       [4.3335021e-03, 5.2602943e-02, 9.4306356e-01, 5.0179386e-18],
       [8.2089363e-10, 6.4054498e-11, 1.0000000e+00, 0.0000000e+00]],
      dtype=float32)

In [121]:
y_pred_cnn = np.argmax(pred_cnn, axis=1)
print("Predicted labels: ", y_pred_cnn)
print("True labels:      ", y_test_cnn)

Predicted labels:  [1 2 1 1 0 0 0 0 1 0 1 2 2 2]
True labels:       [0 0 0 1 1 1 1 1 1 2 2 2 3 3]


**Accuracy: $9 / 14$ ($64.28 \%$)**